In [1]:
from pymongo import MongoClient
from ast import literal_eval

import lib
import ml
import nest

In [2]:
client = MongoClient()

In [20]:
jsds = []
for doc in client.mammals.GNC.find():
    jsd = doc['lf']['js']
    jsd = {frozenset(literal_eval(k)) : v for k, v in jsd.items()}
    jsds.append((jsd[frozenset(('Mouse','Opossum'))], doc['_id']))
jsds.sort()
picked = [_id for jsd, _id in jsds[-12:]]

In [21]:
for doc in client.mammals.data.find({'_id' : {'$in' : picked}}):
    if not client.mammals.GNC_picked2.find_one({'_id' : doc['_id']}) is None:
        continue
    doc['tree'] = '(Opossum,(Human,Mouse)internal);'
    fit = ml.rooted(doc, rooted_edges=['Opossum', 'internal'])
    fit['_id'] = doc['_id']
    client.mammals.GNC_picked2.insert_one(fit)

In [22]:
num_fits = 0
for fit in client.mammals.GNC_picked2.find():
    print fit['_id']
    for name, ens in fit['lf']['EN'].items():
        print name, ens, fit['lf']['params']['omega'][name]['bin0']
    if fit['lf']['EN']['internal'] >= 1e-9 and fit['lf']['EN']['Opossum'] >= 1e-9:
        num_fits += 1
print 'got', num_fits, 'ok fits'

ENSG00000005436_mammals
Opossum 1.52095380093 0.252086653239
Mouse 0.26146738845 0.380752632285
internal 0.122091809928 0.252086653239
Human 0.425191586694 0.365825653866
ENSG00000024526_mammals
Opossum 1.1563489829 0.0684463650094
Mouse 0.291002650015 0.298109619323
internal 7.50531666403e-12 0.0684463650094
Human 0.160393729675 0.0836552199917
ENSG00000162614_mammals
Opossum 1.02128936217 0.0993745385695
Mouse 0.23698634589 0.124446852966
internal 0.0412057813838 0.0993745385695
Human 0.211544309934 0.105519831644
ENSG00000172061_mammals
Opossum 0.89314024179 0.0856814252173
Mouse 0.428224923478 0.226509626042
internal 0.19249848222 0.0856814252173
Human 0.240849831515 0.217229370468
ENSG00000185722_mammals
Opossum 0.804158552007 0.0414702448545
Mouse 0.219210066719 0.063935770728
internal 0.0851964866058 0.0414702448545
Human 0.235619842033 0.0467108431148
ENSG00000106443_mammals
Opossum 1.04811893108 0.0862590125799
Mouse 0.147367139847 0.0380240071029
internal 0.0291884947463 0.08

In [23]:
num_sims = 100
for fit in client.mammals.GNC_picked2.find():
    if fit['lf']['EN']['internal'] < 1e-9 or fit['lf']['EN']['Opossum'] < 1e-9:
        continue
    if not client.mammals.data_picked2.find_one({'_id' : { '$regex' : '^'+fit['_id'] }}) is None:
        continue
    lf = nest.inflate_likelihood_function(fit['lf'], model=ml.GNC)
    for i in range(num_sims):
        aln = lf.simulateAlignment(fit['lf']['aln_length'])
        _id = '%s_%02d' % (fit['_id'], i)
        aln = {'_id' : _id, 'aln' : str(aln), 'tree' : '(Opossum, Human, Mouse);'}
        client.mammals.data_picked2.insert_one(aln)

In [24]:
client.mammals.data_picked2.count()

1000

In [25]:
remote_client = MongoClient('mongodb://localhost:27018')

In [27]:
def copy_collection(to_client, from_client, col_name):
    from_col = getattr(from_client.mammals, col_name)
    to_col = getattr(to_client.mammals, col_name)
    to_col.drop()
    for doc in from_col.find():
        to_col.insert_one(doc)

In [28]:
collections_to_copy = ['GNC_picked2', 'data_picked2']
for col_name in collections_to_copy:
    copy_collection(remote_client, client, col_name)

In [29]:
collections_to_copy = ['GNC_picked_sims2', 'Y98_picked_sims2', 'CNFGTR_picked_sims2']
collections_to_copy += [c+'.log' for c in collections_to_copy]
for col_name in collections_to_copy:
    copy_collection(client, remote_client, col_name)

In [30]:
client.mammals.GNC_picked_sims2.count()

1000